In [19]:
import requests, json
import numpy as np
import pandas as pd
import config

In [20]:
#Get fiat current currency exchange rates from Fixer API
response = requests.get(url="http://data.fixer.io/api/latest", params={"access_key": config.fixer_access_key})
symbols = response.json()
df_forex = pd.DataFrame(symbols)
print(df_forex.head())

     success   timestamp base        date       rates
AED     True  1622062624  EUR  2021-05-26    4.478955
AFN     True  1622062624  EUR  2021-05-26   95.601205
ALL     True  1622062624  EUR  2021-05-26  123.003009
AMD     True  1622062624  EUR  2021-05-26  634.896815
ANG     True  1622062624  EUR  2021-05-26    2.189986


In [21]:
#Get crypto currency exchange rates from binance API
base_url = "https://api.binance.com"
endpoint = "/api/v3/ticker/bookTicker"
url = base_url + endpoint
response = requests.get(url)
df_response = pd.DataFrame(response.json())

#Add "spread" column corresponding to spread percentage
df_response['spread'] = ((df_response['askPrice'].astype(float) - df_response['bidPrice'].astype(float))/((df_response['askPrice'].astype(float) + df_response['bidPrice'].astype(float))/2))*100
df_response = df_response.sort_values(by='spread', ascending=False)

#Set symbol column as index
df_crypto = df_response.set_index('symbol')

print(df_crypto.head())

              bidPrice       bidQty         askPrice        askQty      spread
symbol                                                                        
DOTNGN      0.00000000   0.00000000   34592.00000000    0.03864000  200.000000
LTCNGN  76565.00000000   0.14778000  255382.00000000    0.00200000  107.738284
TRXNGN     15.20000000  46.50000000      50.00000000  129.00000000  106.748466
BNBNGN  21700.00000000   0.02500000   48565.00000000    0.05300000   76.467658
XRPNGN    165.00000000   5.15000000     350.00000000    3.20000000   71.844660


In [22]:
BTC_SYMBOLS = ['AUD', 'BRL', 'GBP', 'RUB', 'TRY', 'IDR', 'UAH', 'NGN', 'EUR']
TO_BTC = {'AUD': 'BTCAUD', 'BRL': 'BTCBRL', 'GBP': 'BTCGBP', 'RUB': 'BTCRUB', 'TRY': 'BTCTRY', 'IDR': 'BTCIDRT', 'UAH': 'BTCUAH', 'NGN': 'BTCNGN', 'EUR': 'BTCEUR'}
DOGE_SYMBOLS = ['AUD', 'BRL', 'GBP', 'RUB', 'TRY', 'EUR']
TO_DOGE = {'AUD': 'DOGEAUD', 'BRL': 'DOGEBRL', 'GBP': 'DOGEGBP', 'RUB': 'DOGERUB', 'TRY': 'DOGETRY', 'EUR': 'DOGEEUR'}
ETH_SYMBOLS = ['AUD', 'BRL', 'GBP', 'RUB', 'TRY', 'NGN', 'EUR']
TO_ETH = {'AUD': 'ETHAUD', 'BRL': 'ETHBRL', 'GBP': 'ETHGBP', 'RUB': 'ETHRUB', 'TRY': 'ETHTRY', 'UAH': 'ETHUAH', 'NGN': 'ETHNGN', 'EUR': 'ETHEUR'}
LTC_SYMBOLS = ['BRL', 'GBP', 'RUB', 'EUR']
TO_LTC = {'AUD': 'LTCAUD', 'BRL': 'LTCBRL', 'GBP': 'LTCGBP', 'RUB': 'LTCRUB', 'TRY': 'LTCTRY', 'EUR': 'LTCEUR'}
XRP_SYMBOLS = ['AUD', 'BRL', 'GBP', 'RUB', 'TRY', 'EUR']
TO_XRP = {'AUD': 'XRPAUD', 'BRL': 'XRPBRL', 'GBP': 'XRPGBP', 'RUB': 'XRPRUB', 'TRY': 'XRPTRY', 'EUR': 'XRPEUR'}

COIN_SYMBOLS = {'BTC': BTC_SYMBOLS, 'ETH': ETH_SYMBOLS, 'DOGE': DOGE_SYMBOLS, 'LTC': LTC_SYMBOLS, 'XRP': XRP_SYMBOLS}
TO_COIN_MAP = {'BTC': TO_BTC, 'ETH': TO_ETH, 'DOGE': TO_DOGE, 'LTC': TO_LTC, 'XRP': TO_XRP}

ROUND_TRIP_DF = pd.DataFrame(columns=['SYMBOL', 'COIN', 'DIRECTION', 'ARB', 'SYMBOL_COIN_SPREAD', 'EURO_COIN_SPREAD'])

for coin in ['BTC', 'ETH', 'DOGE', 'LTC', 'XRP']:
    TO_COIN = TO_COIN_MAP[coin]
    for s in COIN_SYMBOLS[coin]:
        EUR_SYMBOL = float(df_forex.loc[s]['rates'])
        SYMBOL_COIN_SPREAD = df_crypto.loc[TO_COIN[s]]['spread']
        EURO_COIN_SPREAD = df_crypto.loc[TO_COIN['EUR']]['spread']
        #From Euro to Currency to Coin to Euro
        BTC_EUR_BID = float(df_crypto.loc[coin + 'EUR']['bidPrice'])
        SYMBOL_BTC_ASK = float(df_crypto.loc[TO_COIN[s]]['askPrice'])
        round_trip1 = BTC_EUR_BID*EUR_SYMBOL/SYMBOL_BTC_ASK
        ROUND_TRIP_DF = ROUND_TRIP_DF.append({'SYMBOL': s, 'COIN': coin, 'DIRECTION': 'EURO_CURRENCY', 'ARB': round_trip1, 'SYMBOL_COIN_SPREAD': SYMBOL_COIN_SPREAD, 'EURO_COIN_SPREAD': EURO_COIN_SPREAD}, ignore_index=True)
        #From Euro to Coin to Currency to Euro
        BTC_EUR_ASK = float(df_crypto.loc[coin + 'EUR']['askPrice'])
        SYMBOL_BTC_BID = float(df_crypto.loc[TO_COIN[s]]['bidPrice'])
        round_trip2 = (SYMBOL_BTC_BID/EUR_SYMBOL)/BTC_EUR_ASK
        ROUND_TRIP_DF = ROUND_TRIP_DF.append({'SYMBOL': s, 'COIN': coin, 'DIRECTION': 'EURO_COIN', 'ARB': round_trip2, 'SYMBOL_COIN_SPREAD': SYMBOL_COIN_SPREAD, 'EURO_COIN_SPREAD': EURO_COIN_SPREAD}, ignore_index=True)

In [23]:
print(ROUND_TRIP_DF.head())

  SYMBOL COIN      DIRECTION       ARB  SYMBOL_COIN_SPREAD  EURO_COIN_SPREAD
0    AUD  BTC  EURO_CURRENCY  0.996598            0.190469          0.022052
1    AUD  BTC      EURO_COIN  1.001283            0.190469          0.022052
2    BRL  BTC  EURO_CURRENCY  0.992604            0.053329          0.022052
3    BRL  BTC      EURO_COIN  1.006692            0.053329          0.022052
4    GBP  BTC  EURO_CURRENCY  0.999778            0.097215          0.022052
